In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 16, 3, padding=1,bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.MaxPool2d(2, 2),
            nn.Dropout(0.01),

        )
        
        self.conv2 = nn.Sequential(
            nn.Conv2d(16, 32, 3, padding=1,bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(4, 4),
            nn.Dropout(0.01),

        )    
        
        self.conv3 = nn.Sequential(
            nn.Conv2d(32, 50, 3, padding=1,bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(50),
            nn.MaxPool2d(2, 2),
            nn.Dropout(0.01),

        )
      
        self.fc = nn.Sequential(
            nn.Linear(50, 10,bias=False),
        )                
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
      
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        x = F.log_softmax(x, dim=1)
        return x

In [3]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             144
              ReLU-2           [-1, 16, 28, 28]               0
       BatchNorm2d-3           [-1, 16, 28, 28]              32
         MaxPool2d-4           [-1, 16, 14, 14]               0
           Dropout-5           [-1, 16, 14, 14]               0
            Conv2d-6           [-1, 32, 14, 14]           4,608
              ReLU-7           [-1, 32, 14, 14]               0
       BatchNorm2d-8           [-1, 32, 14, 14]              64
         MaxPool2d-9             [-1, 32, 3, 3]               0
          Dropout-10             [-1, 32, 3, 3]               0
           Conv2d-11             [-1, 50, 3, 3]          14,400
             ReLU-12             [-1, 50, 3, 3]               0
      BatchNorm2d-13             [-1, 50, 3, 3]             100
        MaxPool2d-14             [-1, 5

In [4]:
torch.manual_seed(1)
batch_size = 64

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.RandomAffine(degrees=20, translate=(0.1,0.1), scale=(0.9, 1.1)),
                        transforms.ColorJitter(brightness=0.2, contrast=0.2),
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


/home/rogbot/deepvision/lib/python3.8/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [5]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() 
            pred = output.argmax(dim=1, keepdim=True) 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [6]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

torch.set_printoptions(edgeitems=40)

epoch_number = 1
for epoch in range(1, 20):
    print("Epoch Number = ",epoch_number)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    epoch_number += 1

Epoch Number =  1


loss=0.19112752377986908 batch_id=937: 100%|██| 938/938 [00:23<00:00, 39.44it/s]



Test set: Average loss: 0.0512, Accuracy: 9840/10000 (98%)

Epoch Number =  2


loss=0.04713081941008568 batch_id=937: 100%|██| 938/938 [00:23<00:00, 40.23it/s]



Test set: Average loss: 0.0457, Accuracy: 9847/10000 (98%)

Epoch Number =  3


...


Test set: Average loss: 0.0234, Accuracy: 9931/10000 (99%)

Epoch Number =  18


loss=0.03544359281659126 batch_id=937: 100%|██| 938/938 [00:23<00:00, 39.13it/s]



Test set: Average loss: 0.0212, Accuracy: 9936/10000 (99%)

Epoch Number =  19


loss=0.12808680534362793 batch_id=937: 100%|██| 938/938 [00:25<00:00, 37.34it/s]



Test set: Average loss: 0.0229, Accuracy: 9936/10000 (99%)

